In [1]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# importing the dataset
categories = pd.read_csv('data/item_categories.csv')
# to test our model
interactions = pd.read_csv('data/small_matrix.csv')


In [3]:
categories

,video_id,feat
0,0,[8]
1,1,"[27, 9]"
2,2,[9]
3,3,[26]
4,4,[5]
...,...,...
10723,10723,[11]
10724,10724,[2]
10725,10725,[15]
10726,10726,[19]


In [4]:
for i in range (31):
    categories['category_' + str(i)] = 0

categories['feat'] = categories['feat'].apply(ast.literal_eval)
for index, row in categories.iterrows():
    # Get the list of features for the current row
    features = row['feat']

    # Set the corresponding category columns to 1
    for feat in features:
        col_name = f'category_{feat}'
        if col_name in categories.columns:
            categories.at[index, col_name] = 1
categories.drop(columns=['feat'], inplace=True)
categories

,video_id,category_0,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,...,category_21,category_22,category_23,category_24,category_25,category_26,category_27,category_28,category_29,category_30
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,10723,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10724,10724,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10725,10725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10726,10726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# compute the cosine similarity of the categories
similarity = cosine_similarity(categories.iloc[:, 1:])

In [27]:
def get_recommendations(video_id, similarity, categories, top_n=5):
    idx = video_id
    similarity_scores = list(enumerate(similarity[idx]))
     # Sort the similarity scores in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
     # Get the top_n movie indices
    movie_indices = [i[0] for i in similarity_scores[1:top_n+1]]
    return categories['video_id'].iloc[movie_indices]

# Get recommendations for a specific video
# video_id = 0
# recommendations = get_recommendations(video_id, similarity, categories)
# print(f"Recommendations for video ID {video_id}:")
# for rec in recommendations:
#     print(categories[categories["video_id"] == rec])
#     print('------------------')

def get_recommendations_2(videos_id, similarity, categories, top_n=6):
    idx1 = videos_id[0]
    idx2 = videos_id[1]
    idx3 = videos_id[2]
    print(idx1, idx2, idx3)
    similarity_scores1 = list(enumerate(similarity[idx1]))
    similarity_scores2 = list(enumerate(similarity[idx2]))
    similarity_scores3 = list(enumerate(similarity[idx3]))

    similarity_scores1 = sorted(similarity_scores1, key=lambda x: x[1], reverse=True)
    similarity_scores2 = sorted(similarity_scores2, key=lambda x: x[1], reverse=True)
    similarity_scores3 = sorted(similarity_scores3, key=lambda x: x[1], reverse=True)

    top = top_n / 3
    movie_indices1 = [i[0] for i in similarity_scores1[1:int(top)]]
    movie_indices2 = [i[0] for i in similarity_scores2[1:int(top)]]
    movie_indices3 = [i[0] for i in similarity_scores3[1:int(top)]]
    movie_indices = movie_indices1 + movie_indices2 + movie_indices3
    movie_indices = list(set(movie_indices))
    return categories['video_id'].iloc[movie_indices]


In [21]:
# Evaluate the recommendation system using the interactions dataset
# For each user get the top 5 videos their watch the score being the watch_ratio
def generate_test(n, categories, interactions):
    test_df = interactions[interactions['video_id'].isin(categories['video_id'])]
    test_df = interactions[['user_id', 'video_id', 'watch_ratio']]
    test_df = test_df.dropna()
    # print(test_df)
    test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)
    test_df = test_df.set_index(['user_id', 'video_id'])
    return test_df

In [31]:
def evaluate_recommendation_system(df, recommend, similarity, categories, top_n=5):
    precision_list = []
    recall_list = []

    # Group by user
    for user_id, group in df.groupby(level=0):
        # Sort the group by watch_ratio descending
        user_videos = group.sort_values(by='watch_ratio', ascending=False)
        
        if len(user_videos) < 2:
            continue  # Not enough data to evaluate
        
        # Get top video (used as input to model)
        input_video_id = user_videos.index[0][1]
        
        # Get top-4 videos excluding the input one => Ground truth
        ground_truth = user_videos.iloc[1:top_n].index.get_level_values(1).tolist()
        
        # Get top-4 recommended videos from the model
        recommended = recommend(input_video_id, similarity, categories, top_n)
        
        # Remove input_video_id from the recommended list if it's present
        recommended = [vid for vid in recommended if vid != input_video_id][:top_n - 1]
        # print(f"User ID: {user_id}")
        # print(f"Input video ID: {input_video_id}")
        # print(f"Ground truth: {ground_truth}")
        # print(f"Recommended: {recommended}")

        # Compute intersection
        common = set(ground_truth).intersection(set(recommended))

        # Precision = correct / recommended (4)
        precision = len(common) / (top_n - 1)
        precision_list.append(precision)

        # Recall = correct / relevant (min(4, len(ground_truth)))
        recall = len(common) / min(top_n - 1, len(ground_truth))
        recall_list.append(recall)

    # Compute averages
    avg_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0

    return avg_precision, avg_recall

def evaluate_recommendation_system_2(df, recommend, similarity, categories, top_n=6):
    precision_list = []
    recall_list = []

    # Group by user
    for user_id, group in df.groupby(level=0):
        # Sort the group by watch_ratio descending
        user_videos = group.sort_values(by='watch_ratio', ascending=False)
        
        if len(user_videos) < 3:
            continue  # Not enough data to evaluate
        
        # Get top 3 videos (used as input to model)
        id1, id2, id3 = user_videos.index[0][1], user_videos.index[1][1], user_videos.index[2][1]
        input_video_ids = [id1, id2, id3]
        
        # Get top-3 videos excluding the input one => Ground truth
        ground_truth = user_videos.iloc[3:top_n].index.get_level_values(1).tolist()
        
        # Get top-4 recommended videos from the model
        recommended = recommend(input_video_ids, similarity, categories, top_n)
        
        # Remove input_video_id from the recommended list if it's present
        recommended = [vid for vid in recommended if vid not in input_video_ids][:top_n - 1]
        # print(f"User ID: {user_id}")
        # print(f"Input video ID: {input_video_ids}")
        # print(f"Ground truth: {ground_truth}")
        # print(f"Recommended: {recommended}")

        # Compute intersection
        common = set(ground_truth).intersection(set(recommended))

        # Precision = correct / recommended (4)
        precision = len(common) / (top_n - 1)
        precision_list.append(precision)

        # Recall = correct / relevant (min(4, len(ground_truth)))
        recall = len(common) / min(top_n - 1, len(ground_truth))
        recall_list.append(recall)

    # Compute averages
    avg_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0

    return avg_precision, avg_recall


In [23]:
test_df = generate_test(5, categories, interactions)
precision, recal = evaluate_recommendation_system(test_df, get_recommendations, similarity, categories, top_n=5)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recal:.4f}")

test_df = generate_test(20, categories, interactions)
precision, recal = evaluate_recommendation_system(test_df, get_recommendations, similarity, categories, top_n=20)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recal:.4f}")

test_df = generate_test(50, categories, interactions)
precision, recal = evaluate_recommendation_system(test_df, get_recommendations, similarity, categories, top_n=50)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recal:.4f}")

test_df = generate_test(100, categories, interactions)
precision, recal = evaluate_recommendation_system(test_df, get_recommendations, similarity, categories, top_n=100)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recal:.4f}")

/tmp/ipykernel_123262/1308031455.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)


Precision: 0.0009
Recall: 0.0009


/tmp/ipykernel_123262/1308031455.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)


Precision: 0.0038
Recall: 0.0038


/tmp/ipykernel_123262/1308031455.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)


Precision: 0.0074
Recall: 0.0074


/tmp/ipykernel_123262/1308031455.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)


Precision: 0.0158
Recall: 0.0158


In [34]:
test_df = generate_test(15, categories, interactions)
precision, recal = evaluate_recommendation_system_2(test_df, get_recommendations_2, similarity, categories, top_n=15)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recal:.4f}")

/tmp/ipykernel_123262/1308031455.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = test_df.groupby(['user_id']).apply(lambda x: x.nlargest(n, 'watch_ratio')).reset_index(drop=True)


10200 9157 1386
9828 6090 5801
5381 2428 8298
5765 9721 4066
7769 4045 9943
9660 5641 3809
7113 4978 6204
3127 5811 4987
7227 3096 7135
4012 2337 6271
5525 785 8298
3951 9895 5793
103 5525 5834
4220 4109 3684
7244 7157 4123
2119 437 6255
5782 848 7140
581 314 4040
2430 679 8582
2463 7775 2765
7383 154 2130
10377 612 9101
1305 314 6222
4644 7383 154
9976 10341 9790
3599 4712 8228
6985 2025 723
7384 2561 6229
2222 4040 314
2344 6899 7167
3881 2327 6782
4336 7080 2274
8496 8543 211
9059 5027 1507
390 6870 2570
275 6930 8657
4760 7306 7086
3694 573 2765
4065 6090 732
4706 3965 7383
5365 10009 1299
2125 6764 4137
7273 1322 1380
5666 7252 3940
8531 4312 7383
9866 9261 5751
3913 9811 5510
8523 2460 1988
1381 2612 3723
8242 5485 1130
8646 2459 556
2547 1305 5364
2286 2459 2329
7820 4763 671
8516 7260 8673
8366 8532 2339
5027 7097 5878
2437 7101 9884
146 5775 5027
7141 9615 7559
4679 1312 5532
2820 4040 9804
211 6882 3020
2486 392 8419
1414 8311 6829
3590 3672 2130
7531 8786 2944
4704 9985 211
